<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/transformers_asr_pipeline_inconsistencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q transformers datasets huggingface_hub

In [7]:
from transformers import pipeline
from datasets import load_dataset
from huggingface_hub import notebook_login

In [25]:
!transformers-cli env

2023-08-07 08:22:44.292143: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)

Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.31.0
- Platform: Linux-5.15.109+-x86_64-with-glibc2.35
- Python version: 3.10.12
- Huggingface_hub version: 0.16.4
- Safetensors version: 0.3.1
- Accelerate version: not installed
- Accelerate config: not found
- PyTorch version (GPU?): 2.0.1+cu118 (False)
- Tensorflow version (GPU?): 2.12.0 (False)
- Flax version (CPU?/GPU?/TPU?): 0.7.0 (cpu)
- Jax version: 0.4.13
- JaxLib version: 0.4.13
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



In [8]:
notebook_login()

In [9]:
data = load_dataset("mozilla-foundation/common_voice_13_0",
                    "en",
                    split="test",
                    streaming=True)

In [11]:
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-tiny")

In transformers we support two types of `timestamps` one `word` and the other `True` for CTC and Whisper models.

This leads to inconsistent behavior across different types of models.

# Whisper w/ `return_timestamps=True`

In [20]:
pipe(
    {"raw" : next(iter(data))["audio"]["array"],
     "sampling_rate": next(iter(data))["audio"]["sampling_rate"]},
    return_timestamps=True)

Reading metadata...: 16372it [00:00, 43642.13it/s]
Reading metadata...: 16372it [00:00, 67671.33it/s]


{'text': ' Joel Keaton disapproved of films and buster also had reservations about the medium.',
 'chunks': [{'timestamp': (0.0, 5.44),
   'text': ' Joel Keaton disapproved of films and buster also had reservations about the medium.'}]}

# Whisper w/ `return_timestamps=word`

In [21]:
pipe(
    {"raw" : next(iter(data))["audio"]["array"],
     "sampling_rate": next(iter(data))["audio"]["sampling_rate"]},
    return_timestamps="word")

Reading metadata...: 16372it [00:00, 32642.45it/s]
Reading metadata...: 16372it [00:00, 40001.79it/s]


ValueError: ignored

In [22]:
pipe = pipeline(
    "automatic-speech-recognition",
    model="facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2 w/ `return_timestamp=True`

In [23]:
pipe(
    {"raw" : next(iter(data))["audio"]["array"],
     "sampling_rate": next(iter(data))["audio"]["sampling_rate"]},
    return_timestamps=True)

Reading metadata...: 16372it [00:00, 19488.75it/s]
Reading metadata...: 16372it [00:00, 21142.74it/s]


KeyError: ignored

Wav2Vec2 w/ `return_timestamp="word"`

In [24]:
pipe(
    {"raw" : next(iter(data))["audio"]["array"],
     "sampling_rate": next(iter(data))["audio"]["sampling_rate"]},
    return_timestamps="word")

Reading metadata...: 16372it [00:00, 33648.12it/s]
Reading metadata...: 16372it [00:00, 26051.89it/s]


{'text': 'JO KEEPSAN DISAPPROVED OF THONES AND BUSTER ALSO HAD HESERVATIONS ABOUT THE MEDIUM',
 'chunks': [{'text': 'JO', 'timestamp': (0.76, 0.88)},
  {'text': 'KEEPSAN', 'timestamp': (1.06, 1.42)},
  {'text': 'DISAPPROVED', 'timestamp': (1.5, 2.08)},
  {'text': 'OF', 'timestamp': (2.18, 2.22)},
  {'text': 'THONES', 'timestamp': (2.36, 2.68)},
  {'text': 'AND', 'timestamp': (2.8, 2.88)},
  {'text': 'BUSTER', 'timestamp': (2.98, 3.32)},
  {'text': 'ALSO', 'timestamp': (3.42, 3.62)},
  {'text': 'HAD', 'timestamp': (3.7, 3.84)},
  {'text': 'HESERVATIONS', 'timestamp': (3.9, 4.58)},
  {'text': 'ABOUT', 'timestamp': (4.68, 4.9)},
  {'text': 'THE', 'timestamp': (4.94, 5.02)},
  {'text': 'MEDIUM', 'timestamp': (5.08, 5.38)}]}